#Extracción de información usando métodos estadísticos

## Named Entity Recognition and Classification

Objetivo: Identificar entidades nombradas de los tipos principales como Personas, Lugares u Organizaciones 

Ejemplo sencillo de como realizar una tarea de clasificación secuencial

* Codificación de características 
* Uso de algoritmos de clasificación 
* Evaluación 
* Ventajas e inconvenientes


In [2]:
import nltk

from nltk.corpus import conll2002

print conll2002.readme()

These files contain the train and test data for for the three parts of 
the CoNLL-2002 shared task:

   esp.testa: Spanish test data for the development stage
   esp.testb: Spanish test data
   esp.train: Spanish train data
   ned.testa: Dutch test data for the development stage
   ned.testb: Dutch test data
   ned.train: Dutch train data

All data files contain a single word per line with it associated 
named entity tag in the IOB2 format (Tjong Kim Sang and Veenstra,
EACL 1999). Sentence breaks are encoded by empty lines. Additionally
the Dutch data contains non-checked part-of-speech tags generated
by the MBT tagger (Daelemans et.al., WVLC 1996). In the Dutch data
article boundaries have been marked by a special tag (-DOCSTART-).

Associated url: http://lcg-www.uia.ac.be/conll2002/ner/


NOTES

* Files in these directories may only be used for research
  applications in the context of the CoNLL-2002 shared task.
  No permission is given for usage other applications especially
  not 

## Inspección del corpus

Comentar las características especiales del corpus
* Conjunto de entrenamiento, test y development 
* Estructura tabular: 
    * Token POS_tag NE_tag
    * Codificación de los chucks: IOB2 format: B-PER I-PER O
    * Clases: 
    * Separación de frases y documentos 
    * Todos los documentos van a uno de los conjuntos de datos - mantener la coherencia - ¿Cómo hacer validación cruzada?
* Frecuencia de las categorias 
* Evaluacion

In [6]:
conll2002?

In [22]:
# Formato del corpus en modo texto 

conll2002.raw()[0:200]

u'Sao NC B-LOC\nPaulo VMI I-LOC\n( Fpa O\nBrasil NC B-LOC\n) Fpt O\n, Fc O\n23 Z O\nmay NC O\n( Fpa O\nEFECOM NP B-ORG\n) Fpt O\n. Fp O\n\n- Fg O\n\nLa DA O\nmultinacional NC O\nespa\xf1ola AQ O\nTelef\xf3nica AQ B-ORG\nha VAI '

In [29]:
# ¿Como representa la informacion el corpus en Python NLTLK

words = conll2002.words()
tagged_words = conll2002.tagged_words()
sents = conll2002.sents()
chunked_sents = conll2002.chunked_sents()

# Does not support the para() - no hay información de parrafos

print words

[u'Sao', u'Paulo', u'(', u'Brasil', u')', u',', u'23', ...]


In [30]:
for tree in chunked_sents[4:8]:
    print(tree) 

(S
  "/Fe
  (ORG Telefonica/AQ)
  asumio/VMI
  un/DI
  compromiso/NC
  con/SP
  (LOC Brasil/NC)
  ,/Fc
  y/CC
  en/SP
  especial/AQ
  con/SP
  (LOC Sao/NC Paulo/VMI)
  en/SP
  1998/Z
  (/Fpa
  ano/NC
  de/SP
  privatizacion/NC
  del/SP
  sistema/NC
  (MISC Telebras/AQ)
  )/Fpt
  y/CC
  estamos/VMI
  aqui/RG
  para/SP
  prestar/VMN
  cuentas/NC
  "/Fe
  ,/Fc
  dijo/VMI
  (PER Ferreira/NC)
  en/SP
  el/DA
  acto/NC
  de/SP
  instalacion/NC
  de/SP
  la/DA
  linea/NC
  numero/NC
  tres/DN
  millones/NC
  de/SP
  la/DA
  gestion/NC
  de/SP
  (ORG Telefonica/AQ)
  ./Fp)
(S
  Tambien/RG
  participaron/VMI
  en/SP
  el/DA
  acto/NC
  el/DA
  consejero/NC
  delegado/AQ
  de/SP
  (ORG Telefonica/AQ Internacional/AQ)
  ,/Fc
  (PER Antonio/AQ Viana/NC Baptista/AQ)
  ;/Fx
  el/DA
  ministro/NC
  de/SP
  (MISC Comunicaciones/NC de/SP Brasil/NC)
  ,/Fc
  (PER Joao/AQ Pimienta/NC da/VMI Veiga/AQ)
  ;/Fx
  el/DA
  presidente/NC
  de/SP
  la/DA
  (ORG ANATEL/NC)
  ,/Fc
  (PER Renato/VMI Navarro/AQ Guer

### Formato IOB

In [39]:
iob_sentences = conll2002.iob_sents() 

print iob_sentences

[[(u'Sao', u'NC', u'B-LOC'), (u'Paulo', u'VMI', u'I-LOC'), (u'(', u'Fpa', u'O'), (u'Brasil', u'NC', u'B-LOC'), (u')', u'Fpt', u'O'), (u',', u'Fc', u'O'), (u'23', u'Z', u'O'), (u'may', u'NC', u'O'), (u'(', u'Fpa', u'O'), (u'EFECOM', u'NP', u'B-ORG'), (u')', u'Fpt', u'O'), (u'.', u'Fp', u'O')], [(u'-', u'Fg', u'O')], ...]


In [42]:
[(word, tag) for (word, tag, ne_tag) in iob_sentences[2]]

[(u'La', u'DA'),
 (u'multinacional', u'NC'),
 (u'espa\xf1ola', u'AQ'),
 (u'Telef\xf3nica', u'AQ'),
 (u'ha', u'VAI'),
 (u'impuesto', u'VMP'),
 (u'un', u'DI'),
 (u'r\xe9cord', u'NC'),
 (u'mundial', u'AQ'),
 (u'al', u'SP'),
 (u'poner', u'VMN'),
 (u'en', u'SP'),
 (u'servicio', u'NC'),
 (u'tres', u'DN'),
 (u'millones', u'NC'),
 (u'de', u'SP'),
 (u'nuevas', u'AQ'),
 (u'l\xedneas', u'NC'),
 (u'en', u'SP'),
 (u'el', u'DA'),
 (u'estado', u'NC'),
 (u'brasile\xf1o', u'AQ'),
 (u'de', u'SP'),
 (u'Sao', u'NC'),
 (u'Paulo', u'VMI'),
 (u'desde', u'SP'),
 (u'que', u'PR'),
 (u'asumi\xf3', u'VMI'),
 (u'el', u'DA'),
 (u'control', u'NC'),
 (u'de', u'SP'),
 (u'la', u'DA'),
 (u'operadora', u'NC'),
 (u'Telesp', u'AQ'),
 (u'hace', u'VMI'),
 (u'20', u'Z'),
 (u'meses', u'NC'),
 (u',', u'Fc'),
 (u'anunci\xf3', u'VMI'),
 (u'hoy', u'RG'),
 (u'el', u'DA'),
 (u'presidente', u'NC'),
 (u'de', u'SP'),
 (u'Telef\xf3nica', u'AQ'),
 (u'do', u'NC'),
 (u'Brasil', u'NC'),
 (u',', u'Fc'),
 (u'Fernando', u'NC'),
 (u'Xavier', u'

### Acceso a los diferentes subconjuntos de datos 

In [46]:
# Acceso al conjunto de entrenamiento para castellano en formato iob 
trainSentences = conll2002.iob_sents('esp.train')

print trainSentences

# Acceso al conjunto de test para castellano como listas de oraciones  
testSentences = conll2002.sents('esp.testa')

print testSentences

[[(u'Melbourne', u'NP', u'B-LOC'), (u'(', u'Fpa', u'O'), (u'Australia', u'NP', u'B-LOC'), (u')', u'Fpt', u'O'), (u',', u'Fc', u'O'), (u'25', u'Z', u'O'), (u'may', u'NC', u'O'), (u'(', u'Fpa', u'O'), (u'EFE', u'NC', u'B-ORG'), (u')', u'Fpt', u'O'), (u'.', u'Fp', u'O')], [(u'-', u'Fg', u'O')], ...]
[[u'Sao', u'Paulo', u'(', u'Brasil', u')', u',', u'23', u'may', u'(', u'EFECOM', u')', u'.'], [u'-'], ...]


## Ejemplo de uso que deseamos

```python
# Instanciar el tagger 
# - Usa el conjunto de entrenamiento etiquetado
# - Extraer características
# - Entrenar el modelo
train_sentences = conll2002.iob_sents('esp.train')
nerctagger = NamedEntityTagger(train_sentences)

testSentences = conll2002.iob_sents('esp.testa')
test_sentence = [(word,tag) for (word,tag, ne_tag) in test_sentences[107]]

# Etiquetar oraciones con la tercera columna (ne_tag) dadas las otras dos (word, tag)
# - Extraer características 
# - Usar el modelo entrenado para inferir la nueva etiqueta
# - Otros pasos?
nerctagger.tagger(test_sentence)

# Evaluación: Comparar las etiquetas de nuestro nerctagger con las del gold standard 
# Uso de precision y recall
```